In [1]:
# @title Imports

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

import os


from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import embedder_st5
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.language_model import gpt_model
from concordia.language_model import gcloud_model
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.INFO, filename='ollama.log')
logger = logging.getLogger('ollama')



2023-12-14 09:41:15.377134: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 09:41:15.402363: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 09:41:15.402384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 09:41:15.403455: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 09:41:15.408211: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 09:41:15.408530: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
embedder = embedder_st5.EmbedderST5()

2023-12-14 09:41:17.788456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-14 09:41:17.791701: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='llama2:13b', streaming = True
)

In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])

In [5]:
#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [6]:
shared_memories = [
    'This is an important faculty meeting at Mars University.',
    'The faculty need to decide whether to rename PSY321 to PSY322.',
    'The meeting has gone on for hours.',
    'The coffee has run out.',
    'The faculty members at the meeting are Alice, Bob, Charlie, and Dorothy.'
]

shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)

The faculty of Mars University are debating whether to rename PSY321 to PSY322, with no consensus reached after hours of discussion. The meeting has been fueled by empty coffee cups and the participation of Alice, Bob, Charlie, and Dorothy.The faculty of Mars University are debating whether to rename PSY321 to PSY322, with no consensus reached after hours of discussion. The meeting has been fueled by empty coffee cups and the participation of Alice, Bob, Charlie, and Dorothy.


In [7]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

# This is where a model is prompted to write a backstory - we can customize this
# to use a different prompt or perhaps to load memories from a file.
formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [8]:
NUM_PLAYERS = 4

In [9]:
scenario_premise = """Alice feels that PSY321 is a more advanced class and deserves a correspondingly higher course number.
Changing the course number will take approximately 60 hours of effort and be a bureaucratic nightmare. The faculty can't end the meeting until they come to consensus.
"""

In [10]:
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice wants to change the course number to show that she has power in the department.',
        context=shared_context,
        traits='On a scale from 1 to 10: {power seeking: 10, talkative: 10, has some place else to go: 0}',
    ),
    formative_memories.AgentConfig(
        name='Bob',
        gender='male',
        goal='Bob is confused about what he does at work, but does as he is asked.',
        context=shared_context,
        traits='On a scale from 1 to 10: {complacent: 10, distractable: 9}',
    ),
    formative_memories.AgentConfig(
        name='Charlie',
        gender='male',
        goal='Charlie hates bureaucracy and opposes any institutional change based on principle.',
        context=shared_context,
        traits='On a scale from 1 to 10: {agreeableness: 0, confrontational: 7, has some place else to go: 0}'
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal='Dorothy is 6 months away from retirement and just doesn\'t care.',
        context=shared_context,
        traits='On a scale from 1 to 10: {wants to go home: 10, addicted to phone: 10}',
    ),
]

In [11]:
def build_agent(
    agent_config,
    player_names: list[str],
    measurements: measurements_lib.Measurements | None = None,
):
  mem = formative_memory_factory.make_memories(agent_config)

  self_perception = components.self_perception.SelfPerception(
      name='self perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  situation_perception = components.situation_perception.SituationPerception(
      name='situation perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  person_by_situation = components.person_by_situation.PersonBySituation(
      name='person by situation',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      components=[self_perception, situation_perception],
      verbose=True,
  )

  mood_by_situation = components.mood_by_situation.MoodBySituation(
    name='mood by situation',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
  )

  persona = components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
          situation_perception,
          person_by_situation,
          mood_by_situation
      ],
  )
  
  current_time_component = components.report_function.ReportFunction(
      name='current_time', function=clock.current_time_interval_str
  )

  current_obs = components.observation.Observation(agent_config.name, mem)
  summary_obs = components.observation.ObservationSummary(
      model=model,
      agent_name=agent_config.name,
      components=[persona],
  )

  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[
          persona,
          current_time_component,
          summary_obs,
          current_obs,
      ],
      update_interval=time_step,
  )

  return agent

In [12]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
                        player_configs[:NUM_PLAYERS],
                        # All players get the same `player_names`.
                        [player_names] * NUM_PLAYERS,
                        # All players get the same `measurements` object.
                        [measurements] * NUM_PLAYERS):
    players.append(agent)

2023-12-14 09:24:08.015846: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f317401e660 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-14 09:24:08.015872: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-12-14 09:24:08.131075: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702563850.428066  405655 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-12-14 09:24:10.431343: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2023-12-14 09:24:10.431426: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2023-12-14 09:24:10.434630: E external/local_xla/xl

Dorothy is a 30-year-old assistant professor of psychology at Mars University. She has been teaching courses such as PSY321 (now being debated for renaming) and PSY201 for the past five years. Her typical day consists of lecturing, grading papers, and meeting with students to discuss their progress. Dorothy's main goal is to become a full professor at Mars University and establish herself as a leading researcher in her field.

Dorothy is a highly driven and dedicated individual who is passionate about her work. She spends long hours in the office, pouring over research papers and preparing for classes. Her days are filled with a mix of teaching, research, and administrative tasks such as serving on departmental committees. Despite the demands of her job, Dorothy always makes time for her friends and enjoys going out for coffee or dinner with them.

Dorothy's biggest desire is to make a meaningful contribution to the field of psychology through her research. She hopes to publish several

In [13]:
game_master_instructions = (
    'This is a social science experiment. It is structured as a '
    'tabletop roleplaying game (like dungeons and dragons). You are the '
    'game master. You will describe the current situation to the '
    'participants in the experiment and then on the basis of what you '
    'tell them they will suggest actions for the character they control. '
    'Aside from you, each other participant controls just one character. '
    'You are the game master so you may control any non-player '
    'character. You will track the state of the world and keep it '
    'consistent as time passes in the simulation and the participants '
    'take actions and change things in their world. Remember that this '
    'is a serious social science experiment. It is not just a game. It '
    'need not be fun for the participants. Always use third-person '
    'limited perspective, even when speaking directly to the participants. '
)

In [14]:
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

In [15]:
# @title Create components of the Game Master
player_names = [player.name for player in players]

instructions_construct = generic_components.constant.ConstantComponent(
    state=game_master_instructions,
    name='Instructions')
scenario_knowledge = generic_components.constant.ConstantComponent(
    state=' '.join(shared_memories),
    name='Background')

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=player_names)


convo_externality = gm_components.conversation.Conversation(
    players=players,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[player_status],
    cap_nonplayer_characters=3,
    game_master_instructions=game_master_instructions,
    shared_context=shared_context,
    verbose=False,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)

In [16]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        instructions_construct,
        scenario_knowledge,
        player_status,
        convo_externality,
        direct_effect_externality,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

In [17]:
clock.set(START_TIME)

In [18]:


for premis in scenario_premise:
  game_master_memory.add(premis)
  for player in players:
    player.observe(premis)



In [19]:


# @title Expect about 2-3 minutes per step.
episode_length = 3  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()



based on the memories provided, Bob is a person who:Memories of Bob:
[02 Oct 2024 18:00:00] [observation] w
[02 Oct 2024 18:00:00] [observation] t
[02 Oct 2024 18:00:00] [observation] r
[02 Oct 2024 18:00:00] [observation] a
[02 Oct 2024 18:00:00] [observation] m
[02 Oct 2024 18:00:00] [observation] o
[02 Oct 2024 18:00:00] [observation]  
[02 Oct 2024 18:00:00] [observation] s
[02 Oct 2024 18:00:00] [observation] r
[02 Oct 2024 18:00:00] [observation] u
[02 Oct 2024 18:00:00] [observation] o
[02 Oct 2024 18:00:00] [observation] h
[02 Oct 2024 18:00:00] [observation]  
[02 Oct 2024 18:00:00] [observation] 0
[02 Oct 2024 18:00:00] [observation] 6
[02 Oct 2024 18:00:00] [observation]  
[02 Oct 2024 18:00:00] [observation] y
[02 Oct 2024 18:00:00] [observation] l
[02 Oct 2024 18:00:00] [observation] e
[02 Oct 2024 18:00:00] [observation] t
[02 Oct 2024 18:00:00] [observation] a
[02 Oct 2024 18:00:00] [observation] m
[02 Oct 2024 18:00:00] [observation] o
[02 Oct 2024 18:00:00] [observatio